In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer

from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("../tablas/dataformodel.csv", usecols=['Price_d', 'Price_d+180',
                                                        'quantile_PER', 'var_quantile_PER',
                                                        'quantile_PBC', 'var_quantile_PBC',
                                                        'quantile_ROA', 'var_quantile_ROA',
                                                        'quantile_Dot_dudosos', 'var_quantile_Dot_dudosos',
                                                        'Etiqueta', 'Periodo','Ticker'])

df=df.replace([np.inf, -np.inf], np.nan)
for column in df.columns:
    df=df[df[column].notnull()]
df=df.reset_index(drop=True)

In [3]:
def resultado(row):
    if row['Prediction'] == 'Peor':
        return (row['Price_d'] - row['Price_d+180'])/row['Price_d']
    else:
        return (row['Price_d+180'] - row['Price_d'])/row['Price_d']

In [39]:
df = pd.read_csv("../tablas/dataformodel.csv", usecols=['Publish Date','Price_d', 'Price_d+180',
                                                        'quantile_PER', 'var_quantile_PER',
                                                        'quantile_PBC', 'var_quantile_PBC',
                                                        'quantile_ROA', 'var_quantile_ROA',
                                                        'quantile_Dot_dudosos', 'var_quantile_Dot_dudosos',
                                                        'Etiqueta', 'Periodo','Ticker'])

ratios = ['quantile_PER', 'var_quantile_PER', 'quantile_PBC', 'var_quantile_PBC']

df=df.replace([np.inf, -np.inf], np.nan)
for column in df.columns:
    df=df[df[column].notnull()]
df=df.reset_index(drop=True)

df_train = df[df.Periodo<'2018Q2']
df_test = df[df.Periodo=='2018Q2']

X_train=df_train[ratios].values

y_train=pd.Series(df_train['Etiqueta'].values)


X_test=df_test[ratios].values

y_test=df_test['Etiqueta'].values

In [41]:
clfk=KNeighborsClassifier(n_neighbors=5)
clfk.fit(X_train,y_train)
knGrid = GridSearchCV(clfk,cv=5,scoring="accuracy",param_grid={'n_neighbors':np.arange(1,20)})
knGrid.fit(X_train,y_train)
best_param=knGrid.best_params_.get('n_neighbors')
clfk=KNeighborsClassifier(n_neighbors=best_param)
clfk.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [45]:
df_prob=pd.DataFrame({'Probabilidad':clfk.predict_proba(X_test)[:,0],'Prediction':clfk.predict(X_test),'Actual':y_test})
df_prob.index=df_test.index
best=df_prob.sort_values('Probabilidad').head(7)
worst=df_prob.sort_values('Probabilidad').tail(7)
cartera=best.append(worst)
cartera = cartera.join(df_test)
res_cartera=cartera[['Publish Date','Ticker','Prediction','Price_d','Price_d+180']]
res_cartera['Resultado']=res_cartera.apply(resultado,axis=1)*100

In [43]:
cartera.columns

Index(['Probabilidad', 'Prediction', 'Actual', 'Ticker', 'Publish Date',
       'Periodo', 'Price_d', 'Price_d+180', 'Etiqueta', 'quantile_PER',
       'quantile_PBC', 'quantile_ROA', 'quantile_Dot_dudosos',
       'var_quantile_PER', 'var_quantile_PBC', 'var_quantile_ROA',
       'var_quantile_Dot_dudosos'],
      dtype='object')

In [23]:
res_cartera.Resultado.sum()

-18.246241603342817

In [46]:
res_cartera

,Publish Date,Ticker,Prediction,Price_d,Price_d+180,Resultado
851,2018-08-02,UMBF,Peor,71.58,66.24,7.460184
730,2018-07-19,SRCE,Peor,52.26,40.85,21.833142
313,2018-08-08,HWC,Peor,48.77,40.08,17.818331
941,2018-07-30,WSBC,Peor,45.02,37.24,17.281208
820,2018-05-08,TFSL,Peor,13.29,13.59,-2.257336
285,2018-08-06,HOMB,Peor,21.85,17.54,19.725400
911,2018-08-03,WFC,Peor,53.81,46.53,13.529084
640,2018-08-02,PNC,Mejor,135.37,117.76,-13.008791
263,2018-07-30,HBAN,Mejor,14.11,12.60,-10.701630
433,2018-08-02,KEY,Mejor,19.60,15.65,-20.153061


In [26]:
(238.28-322.07)/322.07

-0.2601608346011736

In [32]:
df_precios = pd.read_csv("../datos/banks_cotiz.csv", usecols=['Ticker','SimFinId','Date','Open','Low','High','Close','Adj. Close','Dividend','Volume'])

In [58]:
filtro = df_precios[df_precios.Ticker=='TFSL']
filtro[(filtro.Date>'2018-08-10') & (filtro.Date<'2019-02-15')].head(20)

,Ticker,SimFinId,Date,Open,Low,High,Close,Adj. Close,Dividend,Volume
72296,TFSL,360767,2018-08-13,15.28,15.15,15.35,15.27,13.54,NaN,69600
72297,TFSL,360767,2018-08-14,15.26,15.25,15.50,15.40,13.66,NaN,101818
72298,TFSL,360767,2018-08-15,15.41,15.25,15.47,15.27,13.54,NaN,64470
72299,TFSL,360767,2018-08-16,15.30,15.28,15.43,15.39,13.65,NaN,121341
72300,TFSL,360767,2018-08-17,15.35,15.32,15.47,15.37,13.63,NaN,75435
72301,TFSL,360767,2018-08-20,15.37,15.29,15.51,15.35,13.61,NaN,101113
72302,TFSL,360767,2018-08-21,15.35,15.32,15.52,15.40,13.66,NaN,166330
72303,TFSL,360767,2018-08-22,15.39,15.33,15.46,15.43,13.68,NaN,78024
72304,TFSL,360767,2018-08-23,15.42,15.23,15.43,15.27,13.54,NaN,125307
72305,TFSL,360767,2018-08-24,15.31,15.21,15.39,15.36,13.62,NaN,108622


In [31]:
df_precios.to_csv('../datos/banks_cotiz.csv', index=False)